In [2]:
%config IPCompleter.greedy=True

In [3]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_train

,Id,Label
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0
5,5,1
6,6,0
7,7,1
8,8,0
9,9,0


In [4]:
import pandas as pd
def load_features(col):
    return pd.read_parquet("filtered_features/filtered_feature{}.gzip".format(col))

In [5]:
import pandas as pd

def load_test_features(col):
    return pd.read_parquet("filtered_test_features/filtered_test_feature{}.gzip".format(col))

In [6]:
NUM_OF_COL = 9

In [7]:
SkipCol = {}
# SkipCol[18] = True
SkipCol

{}

In [8]:
y = df_train['Label']
y = y.values
y

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

In [9]:
from sklearn import decomposition
import numpy as np

XTrainAll = []
for col in range(NUM_OF_COL):
    if col in SkipCol:
        continue
    tf = load_features(col)
    XTrainAll.append(tf)
XTrainAll = np.concatenate(XTrainAll,  axis=1)
XTrainAll.shape

(18662, 5585)

In [10]:
np.save('XTrainAll.npy', XTrainAll)

In [11]:
XTestAll = []
for col in range(NUM_OF_COL):
    if col in SkipCol:
        continue
    tf = load_test_features(col)
    tf = tf.replace([np.inf], np.finfo('float32').max).replace([np.inf, -np.inf], np.finfo('float32').min).fillna(0)
    XTestAll.append(tf)
XTestAll = np.concatenate(XTestAll,  axis=1)
XTestAll.shape

(6051, 5585)

In [12]:
np.save('XTestAll.npy', XTestAll)

In [13]:
df_test = pd.read_csv('sample_solution.csv')


In [14]:
y.shape

(18662,)

In [15]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
# clf = GradientBoostingClassifier()
clf = clf.fit(XTrainAll, y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrain = modelSelection.transform(XTrainAll)
XTrain.shape

(18662, 952)

In [16]:
np.save('XTrain.npy', XTrain)

In [17]:
import numpy as np
XTrain = np.load('XTrain.npy', allow_pickle=True)
XTrain.shape

(18662, 952)

In [18]:
XTest = modelSelection.transform(XTestAll)
XTest.shape

(6051, 952)

In [19]:
np.save('XTest.npy', XTest)

In [20]:
XTest = np.load('XTest.npy', allow_pickle=True)
XTest.shape

(6051, 952)

In [21]:
from lightgbm import LGBMModel
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

clf = LGBMModel()
parameters = {'learning_rate': sp_randFloat(),
                  'subsample'    : sp_randFloat(),
                  'n_estimators' : sp_randInt(100, 1000),
                  'max_depth'    : sp_randInt(4, 10) 
                 }

randm = RandomizedSearchCV(estimator=gb_clf, param_distributions = parameters, 
                           cv = 5, n_iter = 10, n_jobs=-1)
randm.fit(XTrain, y)

# Results from Random Search
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

print("\n ========================================================")

gb_clf = randm.best_estimator_

NameError: name 'gb_clf' is not defined

In [22]:
from lightgbm import LGBMClassifier

gb_clf = LGBMClassifier()
gb_clf.fit(XTrain, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [23]:
probs = gb_clf.predict_proba(XTest)
YTest = probs[:, 0]
YTest

array([8.74234326e-04, 3.20600978e-02, 9.98646391e-01, ...,
       9.99433494e-01, 3.14794777e-01, 9.99487590e-01])

In [24]:
np.max(YTest)

0.9996074752189102

In [25]:
train_probs = gb_clf.predict_proba(XTrain)
np.save("lightgbm_train_probs.npy", train_probs[:, 0])
np.save("lightgbm_test_probs.npy", YTest)

In [27]:
np.load("lightgbm_train_probs.npy").shape
np.load("lightgbm_test_probs.npy").shape

(6051,)

In [26]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.001372
1,1,0.082932
2,2,0.998819
3,3,0.037153
4,4,0.808113
5,5,0.989904
6,6,0.999338
7,7,0.953040
8,8,0.001158
9,9,0.011421


In [27]:
df_test.to_csv('test.csv', index=False)